In [ ]:
!pip install -U pip
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0"m
!pip install autogluon  # autogluon==0.2.0

In [ ]:
import os
import gc
import time
import math
import random
import pickle
import numpy as np
import pandas as pd

from autogluon.tabular import TabularDataset, TabularPredictor

import warnings
warnings.simplefilter('ignore')

def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
train_data = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv', index_col=0)
test_data = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv', index_col=0)
idx = test_data.index

train_data = reduce_memory_usage(train_data)
test_data = reduce_memory_usage(test_data)

In [ ]:
# predictor_target = TabularPredictor(label='target', eval_metric="roc_auc", path="AutoGluon8").fit(train_data, time_limit=8*3600, presets='best_quality', verbosity=2, excluded_model_types = ['KNN', 'LR'])
predictor_target = TabularPredictor(label='target', problem_type='regression', eval_metric="root_mean_squared_error", path="AutoGluonRegressor_test").fit(train_data, time_limit=600, presets='best_quality', verbosity=2, excluded_model_types = ['KNN'])

results = predictor_target.fit_summary()
print(results)

In [ ]:
# Use the model to generate predictions
# predictions = predictor_target.predict_proba(test_data)
predictions = predictor_target.predict(test_data, as_pandas=False)

# Save the predictions to a CSV file
submission = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')

submission['target'] = predictions
submission.to_csv("submission.csv", index=False)
submission.head()